In [3]:
import sys
import json
from pathlib import Path

import pandas as pd

sys.path.append(str(Path.cwd().parents[1]))

from training import MODELS
from train_part import ART_DIR
from source.visuals import plot_results_multiple, model2name

### Plot Test Predictions:

In [ ]:
result_preds = dict()
metrics_df = []
for model in MODELS:
    model_art_dir = ART_DIR / f'{model}'
    _metrics_df = pd.read_csv(model_art_dir / 'metrics.csv', index_col=0)
    _metrics_df['model'] = [model.upper(),]
    metrics_df.append(_metrics_df)
    with open(model_art_dir / 'predictions.json', "r") as f:
        pred_dict = json.load(f)
    result_preds[model.upper()] = pred_dict

metrics_df = pd.concat(metrics_df).reset_index(drop=True)
metrics_df['model'] = metrics_df['model'].apply(lambda x: model2name[x])
metrics_df = metrics_df[['model', 'nll_test', 'rmse_test', 'train_time']]
metrics_df.columns = ['Model', 'NLL', 'RMSE', 'Time (s)']

plot_results_multiple(
    result_preds, figsize=(6, 4), save_path='artifacts/predictions.png', show_plot=True, dpi=700)

### Table with Metrics:

In [ ]:
caption = (
    "Trade-off between predictive performance (NLL, RMSE on test)"
    + " and training time for LA-TTKM using VI, CV and, Full GP"
)
print(
    metrics_df.to_latex(
        column_format='||l|c|c|c||',
        caption=caption,
        label="table:robot_dynamics",
        multirow=False,
        index=False,
        float_format="%.1f"
    )
)